In [15]:
from openai import OpenAI
import os
from IPython.display import Markdown, display
import traceback
import io
import os
from contextlib import redirect_stdout, redirect_stderr
from enum import Enum

In [8]:
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key="voc-1142391097160736424292768839a5d4b7a92.77641675")

MODEL = "gpt-4o-mini"

def get_completion(messages):
    try:
        response = client.chat.completions.create(
            model = MODEL,
            messages = messages,
            temperature = 0.7
        )
        return response.choices[0].message.content
    except Exception as err:
        return f"error occrued: {err}"


In [9]:
initial_prompt = f"""
You are an expert Python developer. Please write a Python function based on the following requirements:

We will create a Python function called `process_data` that analyzes numerical data with the following (possibly incomplete) set of requirements:

1. The function should accept a list of numbers and an optional parameter 'mode' that can be 'sum' or 'average' (default should be 'average').
2. If mode is 'sum', return the sum of all numbers.
3. If mode is 'average', return the average (mean) of all numbers.

Example:
```python
process_data([1, 2, 3, 4, 5], mode='average')  # Should return 3.0
process_data([1, 2, 'a', 3], mode='sum')  # Should return 6

Write only the function surrounded by ```python and ``` without any additional explanations or examples.

Example:

```python
def function_name(arguments):
    # function body
```
"""

messages = [{"role": "user", "content": initial_prompt}]
initial_response = get_completion(messages)
print(initial_response)

```python
def process_data(numbers, mode='average'):
    if mode == 'sum':
        return sum(num for num in numbers if isinstance(num, (int, float)))
    elif mode == 'average':
        valid_numbers = [num for num in numbers if isinstance(num, (int, float))]
        return sum(valid_numbers) / len(valid_numbers) if valid_numbers else 0
```


In [10]:
def extract_code(code):
    lines = code.split("\n")
    start = lines.index("```python") + 1
    end = lines.index("```", start)
    return "\n".join(lines[start:end])
initial_code = extract_code(initial_response)
print(initial_code)

def process_data(numbers, mode='average'):
    if mode == 'sum':
        return sum(num for num in numbers if isinstance(num, (int, float)))
    elif mode == 'average':
        valid_numbers = [num for num in numbers if isinstance(num, (int, float))]
        return sum(valid_numbers) / len(valid_numbers) if valid_numbers else 0


In [11]:
test_cases = [
    {"inputs": ([1, 2, 3, 4, 5], "sum"), "expected": 15},
    {"inputs": ([1, 2, 3, 4, 5], "average"), "expected": 3.0},
    {"inputs": ([11, 12, 13, 14, 15], "sum"), "expected": 65},
    {"inputs": ([11, 12, 13, 14, 15], "average"), "expected": 13.0},
    {"inputs": ([1.1, 2.2, 3.3, 4.4, 5.5], "sum"), "expected": 16.5},
    {"inputs": ([1.1, 2.2, 3.3, 4.4, 5.5], "average"), "expected": 3.3},
    {"inputs": ([-1, -2, -3, -4, -5], "sum"), "expected": -15},
    {"inputs": ([-1, -2, -3, -4, -5], "average"), "expected": -3.0},
]

def execute_code(code, test_cases):
    """
    Executes Python code and returns the results of test cases.
    Args:
        code: String containing Python code
        test_cases: List of dictionaries with inputs and expected outputs
    Returns:
        Dictionary containing execution results and test outcomes
    """
    results = {"execution_error": None, "test_results": [], "passed": 0, "failed": 0}

    # Create a namespace for execution
    namespace = {}

    # Capture stdout and stderr
    output_buffer = io.StringIO()

    try:
        with redirect_stdout(output_buffer), redirect_stderr(output_buffer):
            exec(code, namespace)

        # Run test cases
        for i, test in enumerate(test_cases):
            inputs = test["inputs"]
            expected = test["expected"]

            # Execute the function with test inputs
            try:
                if isinstance(inputs, dict):
                    actual = namespace["process_data"](**inputs)
                else:
                    actual = namespace["process_data"](*inputs)

                passed = actual == expected

                if passed:
                    results["passed"] += 1
                else:
                    results["failed"] += 1

                results["test_results"].append(
                    {
                        "test_id": i + 1,
                        "inputs": inputs,
                        "expected": expected,
                        "actual": actual,
                        "passed": passed,
                    }
                )
            except Exception as e:
                # If the error is the expected type, mark as passed
                passed = isinstance(expected, type) and isinstance(e, expected)
                results["test_results"].append(
                    {
                        "test_id": i + 1,
                        "inputs": inputs,
                        "expected": expected,
                        "error": str(e),
                        "passed": passed,
                    }
                )
                if passed:
                    results["passed"] += 1
                else:
                    results["failed"] += 1

    except Exception as e:
        results["execution_error"] = {
            "error_type": type(e).__name__,
            "error_message": str(e),
            "traceback": traceback.format_exc(),
        }

    results["stdout"] = output_buffer.getvalue()
    return results

In [16]:
initial_results = execute_code(initial_code, test_cases)
print(initial_results)

{'execution_error': None, 'test_results': [{'test_id': 1, 'inputs': ([1, 2, 3, 4, 5], 'sum'), 'expected': 15, 'actual': 15, 'passed': True}, {'test_id': 2, 'inputs': ([1, 2, 3, 4, 5], 'average'), 'expected': 3.0, 'actual': 3.0, 'passed': True}, {'test_id': 3, 'inputs': ([11, 12, 13, 14, 15], 'sum'), 'expected': 65, 'actual': 65, 'passed': True}, {'test_id': 4, 'inputs': ([11, 12, 13, 14, 15], 'average'), 'expected': 13.0, 'actual': 13.0, 'passed': True}, {'test_id': 5, 'inputs': ([1.1, 2.2, 3.3, 4.4, 5.5], 'sum'), 'expected': 16.5, 'actual': 16.5, 'passed': True}, {'test_id': 6, 'inputs': ([1.1, 2.2, 3.3, 4.4, 5.5], 'average'), 'expected': 3.3, 'actual': 3.3, 'passed': True}, {'test_id': 7, 'inputs': ([-1, -2, -3, -4, -5], 'sum'), 'expected': -15, 'actual': -15, 'passed': True}, {'test_id': 8, 'inputs': ([-1, -2, -3, -4, -5], 'average'), 'expected': -3.0, 'actual': -3.0, 'passed': True}], 'passed': 8, 'failed': 0, 'stdout': ''}


In [17]:
feedback_prompt = f"""
You are an expert Python developer. You wrote a function based on these requirements:

We will create a Python function called `process_data` that analyzes numerical data with the following (possibly incomplete) set of requirements:

1. The function should accept a list of numbers and an optional parameter 'mode' that can be 'sum' or 'average' (default should be 'average').
2. If mode is 'sum', return the sum of all numbers.
3. If mode is 'average', return the average (mean) of all numbers.

Example:
```python
process_data([1, 2, 3, 4, 5], mode='average')  # Should return 3.0
process_data([1, 2, 'a', 3], mode='sum')  # Should return 6

Write only the function surrounded by ```python and ``` without any additional explanations or examples.

Example:

```python
def function_name(arguments):
    # function body
```

Here is your current implementation:
```python
{initial_code}
```
I've tested your code and here are the results:
{initial_results}

Please improve your code to fix any issues and make sure it passes all test cases.
Write only the improved function without any explanation.
"""

messages = [{"role": "user", "content": feedback_prompt}]

# Get improved code
improved_response = get_completion(messages)

# Extract the improved code
improved_code = extract_code(improved_response)
improved_code

"def process_data(numbers, mode='average'):\n    valid_numbers = [num for num in numbers if isinstance(num, (int, float))]\n    if mode == 'sum':\n        return sum(valid_numbers)\n    elif mode == 'average':\n        return sum(valid_numbers) / len(valid_numbers) if valid_numbers else 0"

In [18]:
improved_results = execute_code(improved_code, test_cases)
print(improved_results)

{'execution_error': None, 'test_results': [{'test_id': 1, 'inputs': ([1, 2, 3, 4, 5], 'sum'), 'expected': 15, 'actual': 15, 'passed': True}, {'test_id': 2, 'inputs': ([1, 2, 3, 4, 5], 'average'), 'expected': 3.0, 'actual': 3.0, 'passed': True}, {'test_id': 3, 'inputs': ([11, 12, 13, 14, 15], 'sum'), 'expected': 65, 'actual': 65, 'passed': True}, {'test_id': 4, 'inputs': ([11, 12, 13, 14, 15], 'average'), 'expected': 13.0, 'actual': 13.0, 'passed': True}, {'test_id': 5, 'inputs': ([1.1, 2.2, 3.3, 4.4, 5.5], 'sum'), 'expected': 16.5, 'actual': 16.5, 'passed': True}, {'test_id': 6, 'inputs': ([1.1, 2.2, 3.3, 4.4, 5.5], 'average'), 'expected': 3.3, 'actual': 3.3, 'passed': True}, {'test_id': 7, 'inputs': ([-1, -2, -3, -4, -5], 'sum'), 'expected': -15, 'actual': -15, 'passed': True}, {'test_id': 8, 'inputs': ([-1, -2, -3, -4, -5], 'average'), 'expected': -3.0, 'actual': -3.0, 'passed': True}], 'passed': 8, 'failed': 0, 'stdout': ''}


In [19]:
print(initial_code)
print('---------')
print(improved_code)

def process_data(numbers, mode='average'):
    if mode == 'sum':
        return sum(num for num in numbers if isinstance(num, (int, float)))
    elif mode == 'average':
        valid_numbers = [num for num in numbers if isinstance(num, (int, float))]
        return sum(valid_numbers) / len(valid_numbers) if valid_numbers else 0
---------
def process_data(numbers, mode='average'):
    valid_numbers = [num for num in numbers if isinstance(num, (int, float))]
    if mode == 'sum':
        return sum(valid_numbers)
    elif mode == 'average':
        return sum(valid_numbers) / len(valid_numbers) if valid_numbers else 0
